In [62]:
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import random
from mpl_toolkits.mplot3d import Axes3D
from bs4 import BeautifulSoup
import urllib
import json
import requests
import datetime
# import dicttoxml
%matplotlib inline

In [80]:
# set variables/initialize data structures once per week at top of hour

dictionary = {}

time = datetime.datetime.now()

week_str = datetime.datetime.strftime(time, "%W")
week_int = int(week_str)

collect_time_str = datetime.datetime.strftime(time, "%m-%d-%H")
collect_hr_int = int(collect_time_str[-2:])

# file = open('wunderground_data/' + collect_time_str + '.json', 'w')

# read initial wunderground json page

# data = get_json()
# add_to_dict(data)


In [65]:
# define functions

def update_time():
    time = datetime.datetime.now()
    
def update_collect_time():
    collect_time_str = datetime.datetime.strftime(time, "%m-%d-%H")
    collect_hr_int = int(collect_time_str[-2:])

def get_json():
    page = requests.get('http://api.wunderground.com/api/f71059696f42912d/forecast/geolookup/conditions/q/MA/Boston.json')
    data = json.loads(page.content)
    return data

def add_to_dict(data):
    key = str(collect_time_str)
    dictionary[key] = data

def write_file(dictionary):
    # writing into file
    json.dump(dictionary, file)


In [ ]:
# main code for one week of data -- NOT FINISHED

init_time = datetime.datetime.now()
init_time_min = int(datetime.datetime.strftime(init_time, "%M"))
end_time = 59
time_not_expired = True

prev_min = init_time_min
while time_not_expired:
    temp_time = datetime.datetime.now()
    temp_time_min = int(datetime.datetime.strftime(temp_time, "%M"))
    time_not_expired = (temp_time_min - init_time_min) != end_time
    prev_min = temp_time_min

mypage = requests.get('http://api.wunderground.com/api/f71059696f42912d/forecast/geolookup/conditions/q/MA/Boston.json')
dictionary = json.loads(mypage.content)
    
print('done')

In [81]:
# 5 minute test

file = open('wunderground_data_test/second_5min_test', 'a')

min = int(datetime.datetime.strftime(time, "%M"))
end_time = 3
time_not_expired = True

prev_min = min
while time_not_expired:
    temp_time = datetime.datetime.now()
    temp_min = int(datetime.datetime.strftime(temp_time, "%M"))
    if prev_min != temp_time_min:
        update_time()
        update_collect_time()
        data = get_json()
        add_to_dict(data)  
    time_not_expired = (temp_min - min) != end_time
    prev_min = temp_min

write_file(dictionary)
print('done')

done


In [74]:
for line in file:
    print(line, end='/n')

UnsupportedOperation: not readable

In [32]:
#for k, v in dictionary.items():
    #print(k, v)

print(dictionary.keys())
dictionary['current_observation'].keys()

dict_keys(['response', 'location', 'current_observation', 'forecast'])


dict_keys(['image', 'display_location', 'observation_location', 'estimated', 'station_id', 'observation_time', 'observation_time_rfc822', 'observation_epoch', 'local_time_rfc822', 'local_epoch', 'local_tz_short', 'local_tz_long', 'local_tz_offset', 'weather', 'temperature_string', 'temp_f', 'temp_c', 'relative_humidity', 'wind_string', 'wind_dir', 'wind_degrees', 'wind_mph', 'wind_gust_mph', 'wind_kph', 'wind_gust_kph', 'pressure_mb', 'pressure_in', 'pressure_trend', 'dewpoint_string', 'dewpoint_f', 'dewpoint_c', 'heat_index_string', 'heat_index_f', 'heat_index_c', 'windchill_string', 'windchill_f', 'windchill_c', 'feelslike_string', 'feelslike_f', 'feelslike_c', 'visibility_mi', 'visibility_km', 'solarradiation', 'UV', 'precip_1hr_string', 'precip_1hr_in', 'precip_1hr_metric', 'precip_today_string', 'precip_today_in', 'precip_today_metric', 'icon', 'icon_url', 'forecast_url', 'history_url', 'ob_url', 'nowcast'])

In [75]:
open?